In [63]:
import pandas as pd
import pickle
import numpy as np
import itertools
from scipy import stats 
import plotly.graph_objects as go

In [41]:
with open('note_list.pkl', 'rb') as f:
    note_list = pickle.load(f)
with open('note_list_transposed.pkl', 'rb') as f:
    note_list_transposed = pickle.load(f)
with open('measure_list.pkl', 'rb') as f:
    measure_list = pickle.load(f)

In [42]:
def KK(mode, transposition):
    """Krumhansl and Kessler (1982) key profiles for major and minor modes"""
    
    major = [6.20, 2.55, 3.45, 2.85, 4.22, 4.57, 2.67, 5.25, 2.45, 3.35, 2.70, 2.70]
    minor = [6.03, 3.35, 3.67, 5.28, 2.58, 3.55, 2.87, 4.80, 4.35, 2.67, 2.50, 3.42]
    
    major_by_fifths = [None] * len(major)
    minor_by_fifths = [None] * len(minor)
    
    #reorder by fifths
    for i in range(len(major)):
        major_by_fifths[(i*7)%12] = major[i]
        minor_by_fifths[((i*7)+3)%12] = minor[i]
    
    #major = list(map(lambda x: x/sum(major), major))
    #minor = list(map(lambda x: x/sum(minor), minor))
    
    if mode == 0:   
        return major_by_fifths[-transposition:]+major[:-transposition]
    elif mode == 1:
        return minor_by_fifths[-transposition:]+minor[:-transposition]
    else:
        print('0 = major, 1 = minor')

In [43]:
def bag_of_notes(df, tpc='tpc'):
    """Input: DataFrame including the columns ['tpc', 'duration'].
    Output: Note distribution."""
    tpcs = df.tpc
    occurring = np.sort(tpcs.unique())
    bag = pd.DataFrame(index=occurring, columns=['count_a', 'count_n', 'duration_a', 'duration_n'])
    GB = df.groupby('tpc')
    bag.count_a = GB.size()
    bag.count_n = bag.count_a / bag.count_a.sum()
    bag.duration_a = GB['duration'].sum().astype(float)
    bag.duration_n = (bag.duration_a / bag.duration_a.sum()).astype(float)
    if tpc != 'tpc':
        names = tpc.split('+')
        note_names = []
        for n in names:
            if n == 'tpc':
                note_names.append(occurring)
            elif n == 'name':
                note_names.append(tpc2name(occurring))
            elif n == 'degree':
                note_names.append(tpc2rn(occurring))
            elif n == 'pc':
                note_names.append(tpc2pc(occurring))
            else:
                logging.warning("Parameter tpc can only be {'tpc', 'name', 'degree', 'pc'} or a combination such as 'tpc+pc' or 'name+degree+tpc'.")
        L = len(note_names)
        if L == 0:
            note_names.append(bag.index)
            L = 1
        if L == 1:
            bag.index = note_names[0]
        else:
            bag.index = [f"{t[0]} ({', '.join(str(e) for e in t[1:])})" for t in zip(*note_names)]
    return bag

In [58]:
def key_slider(dance, m):
    '''Returns the correlation with major and minor key profiles in a 2-measures window starting at m'''
    window = (note_list_transposed.mc >= m) & (note_list_transposed.mc <= m+2)
    bag = bag_of_notes(note_list_transposed[window].loc[dance], 'tpc')
    
    key_profile = pd.DataFrame(index = list(range(12)), columns = ['profile']).fillna(0)
    for i in bag.index:
        key_profile.loc[i] = bag.duration_n.loc[i]
        
    temp = pd.Series(list(itertools.product([0,1], range(12))), index = pd.MultiIndex.from_product([['major', 'minor'], ['C', 'G',  'D',  'A', 'E','B', 'F#/Gb', 'C#/Db','G#/Ab', 'D#/Eb', 'A#/Bb', 'F']]))
    temp = temp.apply(lambda x: KK(x[0], x[1]-3*x[0]))
    temp = temp.apply(lambda x: stats.pearsonr(key_profile.profile, x)[0])
    
    
    return temp

In [59]:
def key_trajectory(dance):
    local_max = []
    trajectory = pd.DataFrame()
    
    for i in range(len(measure_list.loc[dance])-1):
        keys = key_slider(dance,i).copy()
        trajectory = pd.concat([trajectory, keys], axis = 1, ignore_index = True)
        local_max += [keys.idxmax()]
    
    return [trajectory, set(local_max)]

In [104]:
def key_traj_interactive(dance, speed=0.5, xlabel='Bars', ylabel='Correlation', title='Key Correlations'):    
    """ Plots key trajectories and an animated position marker as vertical line

    Args:
        dance: Index of the dance to plot
        speed: Speed of the marker (bars per animation frame)
        xlabel: Label of the x-axis
        ylabel: Label of the y-axis
        title: Plot title

    """


    key_trajectories = key_trajectory(dance)[0][key_trajectory(dance)[0].index.isin(key_trajectory(dance)[1])].transpose()

    # Define the colors of the trajectories
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    maj_colors = ['#1a5276 ', ' #2e86c1 ', ' #7fb3d5 ', ' #154360 ']
    min_colors = ['#FF0010', ' #e74c3c  ', ' #7b241c ', ' #d98880 ']

    n_bars = key_trajectories.shape[0]
    bars = list(range(n_bars))
    data = []

    # Plot the permanent data 
    for idx, key in enumerate(key_trajectories):
        if 'minor' in key:
            color = min_colors[idx%len(colors)]
        else:
            color = maj_colors[idx%len(colors)]
        key_graph1=go.Scatter(x=bars, y=key_trajectories[key],
                              mode="lines",
                              showlegend=False,
                              line=dict(color=color))
        key_graph2=go.Scatter(x=bars, y=key_trajectories[key],
                              name=str(key[1])+' '+str(key[0]),
                              mode="lines",
                              line=dict(color=color))
        data.append(key_graph1)
        data.append(key_graph2)

    # Define the buttons
    updatemenus = [dict(buttons=[dict(args=[None, 
                                            dict(frame=dict(duration=500, redraw=False),
                                                 fromcurrent=True,
                                                 transition=dict(duration=3000, easing='quadratic-in-out'))],
                                      label='Play',
                                      method='animate'),
                                 dict(args=[[None],
                                            dict(frame=dict(duration=0, redraw=False),
                                                 mode='immediate',
                                                 transition=dict(duration=0))],
                                          label='Pause',
                                          method='animate')],
                        type='buttons')]

    #Create the moving frames(i.e. the vertical line that moves from left to right)
    bar_axis = np.arange(0, n_bars, speed)

    frames=[go.Frame(
            data=[go.Scatter(
                x=[bar_axis[k], bar_axis[k]],
                y=[-2, 2],
                mode="lines",
                line=dict(color="black", width=2))
            ]) for k in range(len(bar_axis))]

    # Define the layout of the plot
    layout=go.Layout(width=950, height=600,
                     xaxis=dict(range=[0, max(bars)], autorange=False, zeroline=False),
                     yaxis=dict(range=[-0.4, 1], autorange=False, zeroline=False),
                     xaxis_title=xlabel,
                     yaxis_title=ylabel,
                     title=title,
                     hovermode="closest",
                     updatemenus=updatemenus)

    # Create figure
    fig = go.Figure(data, layout, frames)
    fig.show()

In [107]:
key_traj_interactive(24, title="Key trajectories for Trio D41 Nr.12") 

In [85]:
files.loc[files['gt_mode']=='minor'].head(314)

,id,D,no,dance,path,gt_mode,entropy,duration_entropy,onset_entropy,num_keys,...,TitiTitiTiti_right,TitiTitino_right,TitigiTa_right,TitigiTi_right,TitigiTigitigi_right,TitigiTiti_right,TitigiTitigi_right,Triole_right,nogiTimgi_right,nogiTiti_right
6,7,41,4,menuett,041/D041menuett04a.mscx,minor,0.598936,1.098277,1.944786,6,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,16,41,8,trio,041/D041trio08b.mscx,minor,0.487186,1.268542,2.022451,4,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,18,41,9,trio,041/D041trio09b.mscx,minor,0.559230,0.627309,1.899749,3,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,24,41,12,trio,041/D041trio12b.mscx,minor,0.598641,1.181752,1.893578,3,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,52,128,6,deutscher,128/D128deutscher06.mscx,minor,0.653202,1.006658,1.581356,7,...,1.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,401,969,9,walzer,969/D969walzer09.mscx,minor,0.602801,1.286319,1.445021,5,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
410,411,971,1,deutscher,971/D971deutscher01.mscx,minor,0.589824,0.974953,1.269110,5,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
429,430,977,8,ecossaise,977/D977ecossaise08.mscx,minor,0.613197,1.044776,1.105133,4,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
431,432,979,1,walzer,979/D979walzer01.mscx,minor,0.594023,0.974426,1.348083,6,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
key_trajectories = key_trajectory(24)[0][key_trajectory(24)[0].index.isin(key_trajectory(24)[1])].transpose()

In [91]:
key_trajectories

,"(major, C)","(minor, A)","(minor, G#/Ab)"
0,0.399429,0.641353,0.137016
1,0.404478,0.750187,0.100679
2,0.258818,0.523181,0.199027
3,0.400609,0.767742,0.097453
4,0.315456,0.549681,-0.138842
5,0.605440,0.543689,-0.305232
6,0.657635,0.322509,-0.386849
7,0.862497,0.404246,-0.162565
8,0.910814,0.433166,-0.238019
9,0.504545,-0.151525,0.156091
